### Unstructured Pruning using **PyTorch utils**

In [25]:
!python3 val_prune.py --weights yolov5s.pt --data mycoco100.yaml --img 640 --sparsity 0.3 --device mps

val_prune: data=/Users/iwan/Desktop/3A_HTI/PFE/YOLOv5_Implementation/yolov5/data/mycoco100.yaml, sparsity=0.3, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=mps, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.12.3 torch-2.4.1 MPS

Fusing layers... 
YOLOv5s_v6 summary: 213 layers, 7225885 parameters, 0 gradients
Model pruned to 0.3 global sparsity
val: Scanning /Users/iwan/Desktop/3A_HTI/PFE/YOLOv5_Implementation/datasets/myco
                 Class     Images  Instances          P          R      mAP50   Error decoding JSON from /Users/iwan/Library/Application Support/Ultralytics/settings.json. Starting with an empty dictionary.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/iw

### Pruning YOLOv5 on specific layer

In [9]:
import torch
import torch.nn.utils.prune as prune
from pathlib import Path
import cv2 as cv

In [16]:
model = torch.load("yolov5s.pt")['model']

img = cv.imread("data/images/bus.jpg")
img = cv.resize(img, (640,640))

results = model(img)

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [53]:
def prune_model(model, amount = .3):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

In [51]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Replace 'yolov5s' with your variant
model = torch.load("yolov5s.pt")
torch_model = model["model"]

pruned_torch_model = prune_model(torch_model, amount=.6)

In [54]:
model["model"] = pruned_torch_model
torch.save(model,"yolov5s_pruned.pt")

### PTQ (Post-Training Quantization) using TF-Lite Toolset

In [39]:
import torch
from torchsummary import summary

model = torch.load("yolov5s_prune_0.3.pt")
print(model.parameters())

AttributeError: 'dict' object has no attribute 'parameters'

In [45]:
#Export the model to TF
!python3 export.py --weights yolov5s_prune_0.3.pt --include saved_model --device mps

export: data=data/coco128.yaml, weights=['yolov5s_prune_0.3.pt'], imgsz=[640, 640], batch_size=1, device=mps, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['saved_model']
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.12.3 torch-2.4.1 MPS

Fusing layers... 
YOLOv5s_v6 summary: 213 layers, 7225885 parameters, 0 gradients

PyTorch: starting from yolov5s_prune_0.3.pt with output shape (1, 25200, 85) (27.7 MB)

TensorFlow SavedModel: starting export with tensorflow 2.17.0...
WARNING ⚠️ using Tensorflow 2.17.0 > 2.13.1 might cause issue when exporting the model to tflite https://github.com/ultralytics/yolov5/issues/12489

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  mod

In [46]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv

dataset_dir = "../datasets/mycoco100/images/val2017"

def representative_dataset_gen():
    img_file = [os.path.join(dataset_dir, i) for i in os.listdir(dataset_dir) if i.endswith('.jpg')]

    for img_path in img_file:
        img = cv.imread(img_path)
        # Resize image to the input size expected by the model (e.g., 640x640 for YOLOv5)
        img_resized = cv.resize(img, (640, 640))
        # Normalize image to range [0, 1] if required
        img_normalized = img_resized / 255.0
        # Convert to float32 if the model expects it
        img_input = np.expand_dims(img_normalized, axis=0).astype(np.float32)
        # Yield the preprocessed image
        yield [img_input]

In [47]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("yolov5s_prune_0.3_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

W0000 00:00:1736881125.476808  525644 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1736881125.476858  525644 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: INT8


In [48]:
open('yolov5s_quant.tflite', 'wb').write(tflite_quant_model)

7598712

### PTQ (Post-Training Quantization) using ONNX toolset

In [10]:
model = torch.load("yolov5s_pruned.pt")['model']
model.float()
model.eval() #Puts the model in evaluation mode

dummmy_input = torch.randn(1, 3, 640, 640)
torch.onnx.export(model, dummmy_input, 'yolov5s_pruned.onnx', opset_version=12) #Export the model in onnx format

/Users/iwan/Desktop/3A_HTI/PFE/YOLOv5_Implementation/yolov5/models/yolo.py:268: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
/Users/iwan/Desktop/3A_HTI/PFE/YOLOv5_Implementation/yolov5/models/yolo.py:171: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if visualize:
/Users/iwan/Desktop/3A_HTI/PFE/YOLOv5_Implementation/yolov5/models/yolo.py:167: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future.

In [13]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantized_model = quantize_dynamic('yolov5s_pruned.onnx', 'yolov5s_pruned_quantized.onnx', weight_type=QuantType.QInt8)

In [14]:
from onnx_tf.backend import prepare
import onnx

onnx_model = onnx.load('yolov5s_pruned_quantized.onnx')

tf_rep = prepare(onnx_model)
tf_rep.export_graph('yolov5s_pruned_quantized.pb')

ModuleNotFoundError: No module named 'tensorflow_addons'

### PTQ (Post-Training Quantization) based on IMX-8 datasheet

In [1]:
import tensorflow as tf
import os
import cv2 as cv
import numpy as np

In [2]:
tf_model = "yolov5s_pruned_saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
width = 640
height = 640
channels = 3

dataset_path = "../datasets/coco/images/val2017"

def representative_dataset_generator(folder_path):
    # Iterate over image files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if the file is an image
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue

        # Load and preprocess the image
        try:
            img = cv.imread(file_path, cv.IMREAD_COLOR)
            img = img.resize((width, height))          # Resize to model's input size
            img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]

            # Add batch dimension and yield
            yield [img_array.reshape(1, height, width, channels)]

        except Exception as e:
            print(f"Error processing image {file_path}: {e}")

In [45]:
converter.representative_dataset = representative_dataset_generator(dataset_path)

In [46]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [47]:
tflite_model = converter.convert()

W0000 00:00:1733400583.579077  972825 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1733400583.579091  972825 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.


TypeError: 'generator' object is not callable